In [ ]:
import os
import sqlite3
import time
import csv

In [ ]:
index_db = 'index.db'
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute('''ATTACH DATABASE ? AS other''', (index_db,))

def create_edge_table():
    cur.execute('''CREATE TABLE Edge AS
                     SELECT other.TxIn.addr AS src, other.TxOut.addr AS dst, COUNT(other.TxIn.tx) AS weight
                     FROM other.TxIn
                     INNER JOIN other.TxOut ON other.TxIn.tx = other.TxOut.tx
                     WHERE other.TxIn.addr != 0 AND other.TxOut.addr != 0
                     GROUP BY other.TxIn.addr, other.TxOut.addr;''')
    conn.commit()

def get_edge_count():
    cur.execute('''SELECT COUNT(*) FROM Edge''')
    return cur.fetchone()[0]

def get_edge_cursor():
    cur.execute('''SELECT src, dst, weight
                     FROM Edge;''')
    return cur

In [ ]:
stime = time.time()
create_edge_table()
until = get_edge_count()
now = 0
etime = time.time()
print(f'Progress text prepared during {etime-stime}')

stime = time.time()
edges = list()
weights = list()
with open('edge.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['src', 'dst', 'weight'],
                            quoting=csv.QUOTE_MINIMAL, lineterminator=os.linesep)
    writer.writeheader()
    for src, dst, weight in get_edge_cursor():
        writer.writerow({'src': src,
                         'dst': dst,
                         'weight': weight})
        now = now + 1
        print(f'{now/until*100:010.6f}', end='\r')
print()
etime = time.time()
print(f'Add Edges Done during {etime-stime}')

In [ ]:
conn.close()